### Task_6

In [0]:
from pyspark.sql.functions import concat_ws, col, udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
deliveries_df = spark.read.option("header", True).csv("/FileStore/tables/ipl_sample_deliveries.csv")

In [0]:
deliveries_df = deliveries_df.withColumn("ball_summary",
    concat_ws(" ", col("bowler"), col("batter"), col("total_runs"))
)

In [0]:
info_schema = StructType([
    StructField("Bowler", StringType(), True),
    StructField("Batsman", StringType(), True),
    StructField("Runs", IntegerType(), True)
])

In [0]:
def parse_ball_summary(summary):
    try:
        parts = summary.strip().split()
        if len(parts) >= 3:
            bowler = parts[0]
            batsman = parts[1]
            runs = parts[2]
            return (bowler, batsman, int(runs)) if runs.isdigit() else (bowler, batsman, 0)
    except Exception:
        pass
    return (None, None, None)

In [0]:
parse_summary_udf = udf(parse_ball_summary, info_schema)
parsed_df = deliveries_df.withColumn("parsed_info", parse_summary_udf(col("ball_summary")))

In [0]:
parsed_df.select("ball_summary", "parsed_info").display()

ball_summary,parsed_info
P Kumar SC Ganguly 1,"List(P, Kumar, 0)"
P Kumar BB McCullum 0,"List(P, Kumar, 0)"
P Kumar BB McCullum 1,"List(P, Kumar, 0)"
P Kumar BB McCullum 0,"List(P, Kumar, 0)"
P Kumar BB McCullum 0,"List(P, Kumar, 0)"
P Kumar BB McCullum 1,"List(P, Kumar, 0)"
Z Khan SC Ganguly 2,"List(Z, Khan, 0)"
Z Khan SC Ganguly 0,"List(Z, Khan, 0)"
Z Khan SC Ganguly 4,"List(Z, Khan, 0)"
Z Khan SC Ganguly 6,"List(Z, Khan, 0)"
